In [10]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential 
import pandas as pd
import numpy as np
import pathlib

In [11]:
data_dir='/home/rag-tt/workspace/tactile_images/'
data_dir= pathlib.Path(data_dir)
# Construct the absolute path to the image file
image_path = os.path.join(data_dir, '0', '0.jpg')
image_path

'/home/rag-tt/workspace/tactile_images/0/0.jpg'

In [12]:
def find_no_of_images(obj_id):
    image_dir = os.path.join(data_dir, str(obj_id))
    image_dir= pathlib.Path(image_dir)
    no_of_images= len(list(image_dir.glob('*.jpg')))
    return no_of_images

In [13]:
no_of_images= find_no_of_images(6)
no_of_images

146

In [14]:
label = np.genfromtxt('/home/rag-tt/workspace/tactile_images/7/slip_log.csv', delimiter=',', skip_header=1, usecols=1, dtype=None, encoding=None)
print(label)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1]


In [15]:
# def  import_data(no_of_samples = 50):

#     images = []
#     y = []
#     for obj_id in range(50):
#         no_of_images = find_no_of_images(obj_id)
#         if no_of_images < 40:
#             continue
#         csv_path = os.path.join(data_dir, str(obj_id),'slip_log.csv')
#         label = np.genfromtxt(csv_path, delimiter=',', skip_header=1, usecols=1, dtype=None, encoding=None)
#         y.append(label)
#         for img_id in range(no_of_images):
#             image_path = os.path.join(data_dir, str(obj_id), str(img_id)+ '.jpg')
#             image = cv2.imread(str(image_path))
#             images.append(image)
#     y = np.concatenate(y)
#     y = np.array(y)
#     images= np.array(images)
#     return images, y

In [16]:
# file_paths, y = import_data()

In [17]:
# print('images shape=',images.shape, 'y shape =', y.shape)

In [18]:

# def create_x_y(window_size = 5):
#         X =[]
#         array_shape = (5, 1, 480, 640, 3)
#         import time
#         array_size = len(images)
#         for i in range(array_size-window_size):
#                 row = []
#                 # for a in images[i:i+ window_size]:
#                 #         row.append([a])  
#                 row =[np.array(a) for a in images[i:i+window_size]] # the a is used to wrap every row in a square bracket
#                 row = np.array(row)
#                 X.append(row)  
#         X = np.array(X)
#         y = y[:-window_size]
#         return X, y

In [19]:
def parse_function(filename, label):
    image_string = tf.io.read_file(filename)
    image_decoded = tf.image.decode_jpeg(image_string, channels=3)
    image_resized = tf.image.resize(image_decoded, [480, 640])  # Adjust size as needed
    return image_resized, label

def load_data(data_dir, no_of_samples = 50):
    file_paths = []
    image_paths = []
    y = []
    window_size = 5
    for obj_id in range(no_of_samples):
        no_of_images = find_no_of_images(obj_id)
        if no_of_images < 40:
            continue
        
        csv_path = os.path.join(data_dir, str(obj_id),'slip_log.csv')
        label = np.genfromtxt(csv_path, delimiter=',', skip_header=1, usecols=1, dtype=None, encoding=None)
        y.append(label)
        for img_id in range(no_of_images):
            image_path = os.path.join(data_dir, str(obj_id), str(img_id)+ '.jpg')
            file_paths.append(image_path)

    y = np.concatenate(y)
    y = np.array(y)
    file_paths = np.array(file_paths)
    return file_paths, y

file_paths, labels = load_data(data_dir)


dataset = tf.data.Dataset.from_tensor_slices((file_paths, labels))
dataset = dataset.map(parse_function, num_parallel_calls=tf.data.AUTOTUNE)
batch_size = 32  # Adjust batch size as needed
dataset = dataset.batch(batch_size)
dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

In [20]:
dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 480, 640, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [21]:
# X,y = create_x_y()
# Split dataset into training and validation
dataset_size = len(file_paths)
train_size = int(0.8 * dataset_size)
train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size)

In [22]:
train_dataset

<_TakeDataset element_spec=(TensorSpec(shape=(None, 480, 640, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [23]:
# print('images shape=',X.shape, 'y shape =', y.shape)

In [24]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [25]:
# X_train.shape

In [26]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, TimeDistributed, Conv2D, MaxPooling2D, Flatten, Reshape
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.optimizers import Adam

# Define CNN model
cnn_model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(480, 640, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten()  # Flatten the spatial dimensions
])

cnn_model.summary()
# Define LSTM model
lstm_model = Sequential([
    LSTM(64,input_shape=(5, 144768) ),
    Dense(8, activation='relu'),
    Dense(1, activation='sigmoid'),
])

# Combine CNN and LSTM models
combined_model = Sequential([
    TimeDistributed(cnn_model, input_shape=(480, 640, 3)),  # Apply CNN to each frame in the sequence
    (Reshape((5,144768))),
    lstm_model,
])
cp = ModelCheckpoint('model/', save_best_only=True)
combined_model.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.0001),metrics=['accuracy'])




Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_24 (Conv2D)          (None, 478, 638, 32)      896       
                                                                 
 max_pooling2d_24 (MaxPooli  (None, 239, 319, 32)      0         
 ng2D)                                                           
                                                                 
 conv2d_25 (Conv2D)          (None, 237, 317, 32)      9248      
                                                                 
 max_pooling2d_25 (MaxPooli  (None, 118, 158, 32)      0         
 ng2D)                                                           
                                                                 
 conv2d_26 (Conv2D)          (None, 116, 156, 32)      9248      
                                                                 
 max_pooling2d_26 (MaxPooli  (None, 58, 78, 32)      

ValueError: Exception encountered when calling layer "time_distributed_4" (type TimeDistributed).

Input 0 of layer "sequential_13" is incompatible with the layer: expected shape=(None, 480, 640, 3), found shape=(None, 640, 3)

Call arguments received by layer "time_distributed_4" (type TimeDistributed):
  • inputs=tf.Tensor(shape=(None, 480, 640, 3), dtype=float32)
  • training=None
  • mask=None

In [ ]:


model.fit(train_dataset, epochs=30)

Epoch 1/30


ValueError: in user code:

    File "/home/rag-tt/workspace/deep_slip_detection_env/deep_slip_detection/.venv/lib/python3.8/site-packages/keras/src/engine/training.py", line 1338, in train_function  *
        return step_function(self, iterator)
    File "/home/rag-tt/workspace/deep_slip_detection_env/deep_slip_detection/.venv/lib/python3.8/site-packages/keras/src/engine/training.py", line 1322, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/rag-tt/workspace/deep_slip_detection_env/deep_slip_detection/.venv/lib/python3.8/site-packages/keras/src/engine/training.py", line 1303, in run_step  **
        outputs = model.train_step(data)
    File "/home/rag-tt/workspace/deep_slip_detection_env/deep_slip_detection/.venv/lib/python3.8/site-packages/keras/src/engine/training.py", line 1080, in train_step
        y_pred = self(x, training=True)
    File "/home/rag-tt/workspace/deep_slip_detection_env/deep_slip_detection/.venv/lib/pyth